In [ ]:
import geopandas as gpd
import osmnx as ox
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import contextily as ctx

from shapely.geometry import MultiLineString, LineString, Point
from shapely import wkt
import random
import string
import s3fs
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

import json
import numpy as np

from wetterdienst.provider.dwd.observation import DwdObservationRequest, DwdObservationDataset, DwdObservationPeriod, DwdObservationResolution
from datetime import datetime
from datetime import timedelta
import openturns as ot
import tqdm
from scipy.spatial.distance import cdist


from tesspy import Tessellation

import warnings
warnings.filterwarnings('ignore')

In [ ]:
start_data = datetime.strptime('01/Jan/2019 00:00:00', '%d/%b/%Y %H:%M:%S')
end_data = datetime.strptime('01/Mar/2019 00:00:00', '%d/%b/%Y %H:%M:%S')

In [ ]:
stations_temp = DwdObservationRequest(parameter=[DwdObservationDataset.TEMPERATURE_AIR],
                                resolution=DwdObservationResolution.HOURLY,
                                start_date = start_data,
                                end_date = end_data
                                )

In [ ]:
stations_wind = DwdObservationRequest(parameter=[DwdObservationDataset.WIND],
                                resolution=DwdObservationResolution.HOURLY,
                                start_date = start_data,
                                end_date = end_data
                                )

In [ ]:
stations_peripitation = DwdObservationRequest(parameter=[DwdObservationDataset.PRECIPITATION],
                                resolution=DwdObservationResolution.HOURLY,
                                start_date = start_data,
                                end_date = end_data
                                )

In [ ]:
df_all_station = stations_temp.all().df

In [ ]:
temp_data = stations_temp.filter_by_station_id(station_id=df_all_station.station_id.unique()).values.all().df

In [ ]:
wind_data = stations_wind.filter_by_station_id(station_id=df_all_station.station_id.unique()).values.all().df

In [ ]:
peripitation_data = stations_peripitation.filter_by_station_id(station_id=df_all_station.station_id.unique()).values.all().df

In [ ]:
temp_data = temp_data[temp_data.parameter=='temperature_air_mean_200']

In [ ]:
wind_data = wind_data[wind_data.parameter=='wind_speed']

In [ ]:
peripitation_data = peripitation_data[peripitation_data.parameter=='precipitation_height']

In [ ]:
temp_data = temp_data.merge(df_all_station[["station_id", "latitude","longitude"]], left_on='station_id', right_on='station_id')

In [ ]:
wind_data = wind_data.merge(df_all_station[["station_id", "latitude","longitude"]], left_on='station_id', right_on='station_id')

In [ ]:
peripitation_data = peripitation_data.merge(df_all_station[["station_id", "latitude","longitude"]], left_on='station_id', right_on='station_id')

In [ ]:
#temp_data.drop(columns=["dataset", "parameter", "quality"], inplace=True)
#wind_data.drop(columns=["dataset", "parameter", "quality"], inplace=True)
peripitation_data.drop(columns=["dataset", "parameter", "quality"], inplace=True)

## Problem: incomplete weather data
### solution: 

- create DF with all stations with station_id, timestamp, lat, lon, temp_value, wind_value, peripitation_value
- merge data based on station_id 
- fill NaN values by restriction on each hour and spatially interpolation data for whole germany for each hour
--> 1417 spatial interpolation

--> shape of overall df will be 888459 x 7

## Interpolation of all temperatur data 

In [ ]:
temperatur_dfs = []

In [ ]:
timestamps = temp_data.date.unique()


for ts in tqdm.tqdm_notebook(timestamps): 
    tmp = temp_data[temp_data.date==ts]
    tmp1 = tmp[tmp.value.notna()]
    tmp2 = tmp[tmp.value.isna()]
    
    coords = ot.Sample(tmp1[["latitude", "longitude"]].to_numpy())
    values = ot.Sample(tmp1[["value"]].to_numpy())
    
    inputDimension = 2
    #basis = ot.ConstantBasisFactory(inputDimension).build()
    #covarianceModel = ot.SquaredExponential([1.]*inputDimension, [1.0])
    
    basis = ot.Basis(0)
    covarianceModel = ot.SphericalModel(2)
    
    algo = ot.KrigingAlgorithm(coords, values, covarianceModel, basis)
    algo.run()
    result = algo.getResult()
    krigingMetamodel = result.getMetaModel()
    
    tmp2["value"] = tmp2.apply(lambda x: round(krigingMetamodel([x["latitude"],x["longitude"]])[0],2), axis=1)
    temperatur_dfs.append(pd.concat([tmp1,tmp2]).sort_values(by="station_id"))
    
temperatur_data = pd.concat(temperatur_dfs)
temperatur_data.drop_duplicates(inplace=True)

In [ ]:
temperatur_data.sort_values(["station_id", "date"], inplace=True)

In [ ]:
temperatur_data.to_csv("all_temperatur_data_20190101_20190301.csv", index=False)

## Interpolation of all wind data 

In [ ]:
wind_dfs = []

In [ ]:
wind_data.head()

In [ ]:
timestamps = wind_data.date.unique()


for ts in tqdm.tqdm_notebook(timestamps): 
    tmp = wind_data[wind_data.date==ts]
    tmp1 = tmp[tmp.value.notna()]
    tmp2 = tmp[tmp.value.isna()]
    
    coords = ot.Sample(tmp1[["latitude", "longitude"]].to_numpy())
    values = ot.Sample(tmp1[["value"]].to_numpy())

    
    basis = ot.Basis(0)
    covarianceModel = ot.SphericalModel(2)
    
    algo = ot.KrigingAlgorithm(coords, values, covarianceModel, basis)
    algo.run()
    result = algo.getResult()
    krigingMetamodel = result.getMetaModel()
    
    tmp2["value"] = tmp2.apply(lambda x: round(krigingMetamodel([x["latitude"],x["longitude"]])[0],2), axis=1)
    wind_dfs.append(pd.concat([tmp1,tmp2]).sort_values(by="station_id"))
    
windspeed_data = pd.concat(wind_dfs)
windspeed_data.drop_duplicates(inplace=True)

In [ ]:
#Interpolation error leads to some values < 0 (e.g. -0.5) these values are replaced with 0 
windspeed_data.loc[windspeed_data['value'] < 0, 'value'] = 0

In [ ]:
windspeed_data.sort_values(["station_id", "date"], inplace=True)

In [ ]:
windspeed_data.to_csv("all_windspeed_data_20190101_20190301.csv", index=False)

## Interpolation of all percipitation 

In [ ]:
perci_dfs = []

In [ ]:
timestamps = peripitation_data.date.unique()


for ts in tqdm.tqdm_notebook(timestamps):
    tmp = peripitation_data[peripitation_data.date==ts]
    tmp1 = tmp[tmp.value.notna()]
    tmp2 = tmp[tmp.value.isna()]

    coords = ot.Sample(tmp1[["latitude", "longitude"]].to_numpy())
    values = ot.Sample(tmp1[["value"]].to_numpy())


    basis = ot.Basis(0)
    covarianceModel = ot.SphericalModel(2)
    
    try:
        algo = ot.KrigingAlgorithm(coords, values, covarianceModel, basis)
        algo.run()
        result = algo.getResult()
        krigingMetamodel = result.getMetaModel()

    except:
        pass

    tmp2["value"] = tmp2.apply(lambda x: round(krigingMetamodel([x["latitude"],x["longitude"]])[0],2), axis=1)
    perci_dfs.append(pd.concat([tmp1,tmp2]).sort_values(by="station_id"))
        
        
peripitation_height_data = pd.concat(perci_dfs)
peripitation_height_data.drop_duplicates(inplace=True)

In [ ]:
#Interpolation error leads to some values < 0 (e.g. -0.5) these values are replaced with 0 
peripitation_height_data.loc[peripitation_height_data['value'] < 0, 'value'] = 0

In [ ]:
peripitation_height_data.sort_values(["station_id", "date"], inplace=True)

In [ ]:
peripitation_height_data.to_csv("all_peripitation_height_20190101_20190301.csv", index=False)